<a href="https://colab.research.google.com/github/jett220201/IA_Project/blob/main/ProyectoIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importación de librerias

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef
from sklearn import decomposition
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

Carga del Dataset

In [2]:
Dataframe = pd.read_csv("Dataset.csv",sep=",")
Dataframe.head(10)

,Unnamed: 0,Label,Hu0,Hu1,Hu2,Hu3,Hu4,Hu5,Hu6
0,0,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72
1,1,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72
2,2,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72
3,3,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72
4,4,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72
5,5,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72
6,6,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72
7,7,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72
8,8,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72
9,9,1,0.000681,3.634049e-08,2.419996e-36,5.515569e-36,1.928547e-71,-1.019727e-39,-5.841777e-72


Asignación de características y etiquetas


In [3]:
X = Dataframe[['Hu0','Hu1','Hu2','Hu3','Hu4','Hu5','Hu6']].values #Adquisición de caracteristicas
y = Dataframe['Label'].values #Adquisición de etiquetas 

División del Dataset


In [4]:
#División del dataset en 10% para evaluar desempeño final y 90% para realizar
#entrenamiento y validación cruzada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

Normalización del Dataset

In [5]:
#Normalización de los datos
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

PCA

In [6]:
pca = decomposition.PCA(whiten=False,svd_solver='auto')
pca.fit(X_train)
X_trainPCA = pca.transform(X_train) 

print("Pesos de PCA:", pca.explained_variance_ratio_)

Pesos de PCA: [8.72635364e-01 1.10995931e-01 1.01136287e-02 5.92647050e-03
 3.05479427e-04 2.28169581e-05 3.09662429e-07]


Se puede reducir a 2 dimensiones (0.983631295)

In [7]:
pca = decomposition.PCA(n_components=2)
pca.fit(X_train)

PCA_X_train = pca.transform(X_train)
PCA_X_test = pca.transform(X_test)
PCA_X = pca.transform(X)


In [8]:
#Iteración de hiper parametros para hallar el mejor clasificador KNN usando Grid Search y Cross Validate
parameters_KNN = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 'metric':('manhattan','euclidean','minkowski', 'chebyshev'), 'weights':('uniform', 'distance')}
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters_KNN,scoring="f1_macro")
#clf.fit(X_train, y_train)
clf.fit(PCA_X_train, y_train)
print("Best estimator: " + str(clf.best_estimator_) + "\nwith metrics: " + str(clf.best_params_) + " and score: " + str(clf.best_score_))

Best estimator: KNeighborsClassifier(metric='manhattan', n_neighbors=3)
with metrics: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'} and score: 0.5377339045443461


In [14]:
#Re entrenamiento usando el 90% y validación de resultados usando el 10% y MCC
knn = KNeighborsClassifier(n_neighbors = 3,weights = 'uniform', metric = 'chebyshev')
knn.fit(PCA_X_train, y_train) 
print('MCC = ',matthews_corrcoef(y_test,knn.predict(PCA_X_test)))

MCC =  0.2196969696969697
